# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

SyntaxError: invalid syntax (<ipython-input-3-71a7702f86c3>, line 3)

In [ ]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
import pandas as pd

In [3]:
document_tree = ET.parse( './data/mondial_database.xml' )

In [4]:
#10 countries with the lowest infant mortality rates
d = []
for child in document_tree.getroot():
    if child.find('infant_mortality') is not None:
        d.append({'Country': child.find('name').text, 'mortality': float(child.find('infant_mortality').text)})

In [5]:
data = pd.DataFrame.from_dict(d).sort_values(by='mortality').head(10)
data

,Country,mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [6]:
#10 cities with the largest population
d = []
for element in document_tree.iterfind('country'):
    for subelement in element.getiterator('city'):
        cname = subelement.find('name').text
        for s in subelement.getiterator('population'):
            if 'measured' in s.attrib:
                method = s.attrib['measured']
            else:
                method = "NA"
            d.append({'City': cname, 'year': int(s.attrib['year']), 'population': int(s.text), 'measured': method})

In [7]:
data = pd.DataFrame.from_dict(d)

In [8]:
#10 cities with the largest population - all methodologies and all years
data.sort_values(by='population',ascending=False).head(10)

,City,measured,population,year
3750,Shanghai,census,22315474,2010
3749,Shanghai,census,15758892,2000
2607,Istanbul,admin.,13710512,2012
4398,Delhi,census,12877470,2001
4303,Mumbai,census,12442373,2011
1546,Moskva,estimate,11979529,2013
4302,Mumbai,census,11914398,2001
3746,Beijing,census,11716620,2010
1545,Moskva,census,11612885,2010
8208,São Paulo,census,11152344,2010


In [9]:
#Most recent measurement for each country
pops = data.sort_values(['City','year'],ascending=[True, False]).groupby('City').first()
pops.sort_values(by='population',ascending=False).head(10)

,measured,population,year
City,,,
Shanghai,census,22315474,2010
Istanbul,admin.,13710512,2012
Mumbai,census,12442373,2011
Moskva,estimate,11979529,2013
Beijing,census,11716620,2010
São Paulo,census,11152344,2010
Tianjin,census,11090314,2010
Guangzhou,census,11071424,2010
Delhi,census,11034555,2011


In [10]:
#Look at different measurement types
set(data.measured)

{'NA',
 'admin.',
 'census',
 'count',
 'est.',
 'estimate',
 'prelim. census',
 'projection',
 'survey'}

In [11]:
#10 cities with the largest population - only census
pops[pops.measured == 'census'].sort_values(by='population',ascending=False).head(10)

,measured,population,year
City,,,
Shanghai,census,22315474,2010
Mumbai,census,12442373,2011
Beijing,census,11716620,2010
São Paulo,census,11152344,2010
Tianjin,census,11090314,2010
Guangzhou,census,11071424,2010
Delhi,census,11034555,2011
Shenzhen,census,10358381,2010
Wuhan,census,9785388,2010


In [12]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [13]:
#Get populations and ethnic percentages for all countries
pops = []
ethnic = []
for element in document_tree.iterfind('country'):
    cname = element.find('name').text
    for s in element.getiterator('population'):
        if 'measured' in s.attrib:
            method = s.attrib['measured']
        else:
            method = "NA"
        pops.append({'Country': cname, 'year': int(s.attrib['year']), 'population': int(s.text), 'measured': method})
    for s in element.getiterator('ethnicgroup'):
        ethnic.append({'Country': cname, 'ethnic_pop': s.text, 'pcnt': float(s.attrib['percentage'])})

In [14]:
countries = pd.DataFrame.from_dict(pops).sort_values(['Country','year'], ascending=[True, False])
countryPops = countries.groupby('Country').first().reset_index()

In [15]:
ethnicGroups = pd.DataFrame.from_dict(ethnic)

In [16]:
df = ethnicGroups.merge(countryPops)

In [17]:
df['Size'] = df.pcnt/100 * df.population

In [18]:
df.groupby('ethnic_pop')['Size'].sum().sort_values(ascending=False).head(10)

ethnic_pop
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.304840e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: Size, dtype: float64

In [19]:
#Name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [20]:
#Get country names and codes
c = []
for element in document_tree.iterfind('country'):
    c.append({'Country_name': element.find('name').text, 'Country_code': element.attrib['car_code']})
countries = pd.DataFrame.from_dict(c)

In [21]:
#Longest river
r = []
for element in document_tree.iterfind('river'):
    if element.find('length') is not None:
        length = float(element.find('length').text)
    else:
        length = 0
        
    count = element.attrib['country'].split(" ")
    for c in count:
        r.append({'Country_code': c, 'River_name': element.find('name').text,
             'Length': length})

rivers = pd.DataFrame.from_dict(r).merge(countries)

rivers.sort_values(by='Length',ascending=False).head(1)

,Country_code,Length,River_name,Country_name
299,CO,6448.0,Amazonas,Colombia


In [22]:
#Largest lake
l = []
for element in document_tree.iterfind('lake'):
    if element.find('area') is not None:
        area = float(element.find('area').text)
    else:
        area = 0
    count = element.attrib['country'].split(" ")
    
    for c in count:
        l.append({'Country_code': c, 'Lake_name': element.find('name').text,
             'Area': area})

lakes = pd.DataFrame.from_dict(l).merge(countries)
lakes.sort_values(by='Area',ascending=False).head(1)

,Area,Country_code,Lake_name,Country_name
56,386400.0,R,Caspian Sea,Russia


In [23]:
#airport at highest elevation
a = []
for element in document_tree.iterfind('airport'):
    if element.find('elevation').text is not None:
        elev = float(element.find('elevation').text)
    else:
        elev = 0
    a.append({'Country_code': element.attrib['country'], 'Airport_name': element.find('name').text,
             'Elevation': elev})

airports = pd.DataFrame.from_dict(a).merge(countries)
airports.sort_values(by='Elevation',ascending=False).head(1)

,Airport_name,Country_code,Elevation,Country_name
80,El Alto Intl,BOL,4063.0,Bolivia
